In [11]:
from pathlib import Path
import os
import time
import subprocess
import pandas as pd
from tqdm.notebook import tqdm

# Expande el home y define la ruta base correctamente
folder_path = Path('~/Documents/Universidad/Tercero/PlaniExt/Plani').expanduser()
problem_path = folder_path / "Ejercicio2" / "2_1" / "Problemas"
problem_path.mkdir(parents=True, exist_ok=True)

planner_name = "FF"
planner_cmd = [
    str(folder_path / "planificadores" / "ff"),
    "-o", str(folder_path / "Ejercicio2" / "2_1" / "dron-domain.pddl")
]

sizes, times, steps_list, outputs = [], [], [], []
max_size = 0
size = 2
time_exceeded = False
max_time = 600000  # 60 segundos

In [ ]:


pbar = tqdm(desc=f"Resolviendo con {planner_name}", unit="tamaño")
while not time_exceeded:
    pbar.set_description(f"{planner_name} → tamaño {size}")
    # Genera el problema en el directorio correcto
    gen_cmd = (
        f"python3 {folder_path}/Ejercicio2/2_1/generate_problem.py "
        f"-d 1 -r 1 -k 4 -l {size} -p {size} -c {size} -g {size} -v 0 "
        f"-o {problem_path}"
    )
    os.system(gen_cmd)
    

    pddl_files = sorted(problem_path.glob("*.pddl"), key=os.path.getmtime, reverse=True)
    print(f"Archivos PDDL generados: {len(pddl_files)}")
    if not pddl_files:
        break
    problem_pddl = pddl_files[0]
    start = time.time()
    try:
        res = subprocess.run(planner_cmd + ["-f", str(problem_pddl)],
                             capture_output=True, text=True, timeout=max_time)
        elapsed = time.time() - start
        if res.returncode != 0:
            time_exceeded = True
        else:
            plan_lines = res.stdout.splitlines()
            num_steps = sum(1 for ln in plan_lines if ln.strip() and "(" in ln and not ln.strip().startswith(";"))
            sizes.append(size)
            times.append(elapsed)
            steps_list.append(num_steps)
            outputs.append("\n".join(plan_lines[:50]))
            max_size = size
    except subprocess.TimeoutExpired:
        time_exceeded = True
    # os.remove(problem_pddl)
    size += 1
    pbar.update(1)

pbar.close()
print(f"{planner_name} resolvió hasta tamaño {max_size}")
df_ff = pd.DataFrame({"Tamaño": sizes, "Tiempo": times, "Pasos": steps_list})
df_ff

Resolviendo con FF: 0tamaño [00:00, ?tamaño/s]

FF resolvió hasta tamaño 0


,Tamaño,Tiempo,Pasos
